<a href="https://colab.research.google.com/github/Saianiruth/cxr_foundation/blob/main/notebooks/quick_start_with_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# @title Authenticate with HuggingFace, skip if you have a HF_TOKEN secret

# Authenticate user for HuggingFace if needed. Enter token below if requested.
from huggingface_hub.utils import HfFolder
from huggingface_hub import notebook_login

if HfFolder.get_token() is None:
    from huggingface_hub import notebook_login
    notebook_login()

In [28]:
# Install dependencies
major_version = tf.__version__.rsplit(".", 1)[0]
!pip install tensorflow-text=={major_version} pypng && pip install --no-deps pydicom hcls_imaging_ml_toolkit retrying


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-text
    Found existing installation: tensorflow-text 2.18.1
    Uninstalling tensorflow-text-2.18.1:
      Successfully uninstalled tensorflow-text-2.18.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.1 MB/s eta 0:00:00


In [2]:
import os
import sys
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
import tensorflow as tf
import multiprocessing
# Repository path setup
REPO_PATH = os.path.abspath("cxr-foundation/python")
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)
from clientside.clients import make_hugging_face_client

# Enable resource variables to suppress TensorFlow warnings
tf.compat.v1.enable_resource_variables()


# Initialize CXR client
cxr_client = make_hugging_face_client('cxr_model')

# Load CSV files
df_train = pd.read_csv("data/train.csv")[["Path", "Pleural Effusion", "No Finding"]]
df_valid = pd.read_csv("data/valid.csv")[["Path", "Pleural Effusion", "No Finding"]]

# Ensure paths are formatted correctly
df_train["Path"] = df_train["Path"].apply(lambda x: os.path.join("data", x.split("data/")[-1]))
df_valid["Path"] = df_valid["Path"].apply(lambda x: os.path.join("data", x.split("data/")[-1]))

df_valid.head()

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

,Path,Pleural Effusion,No Finding
0,data/valid/patient64541/study1/view1_frontal.jpg,0,0
1,data/valid/patient64542/study1/view1_frontal.jpg,0,0
2,data/valid/patient64542/study1/view2_lateral.jpg,0,0
3,data/valid/patient64543/study1/view1_frontal.jpg,0,0
4,data/valid/patient64544/study1/view1_frontal.jpg,0,1


In [3]:
df_train.head(10)

,Path,Pleural Effusion,No Finding
0,data/train/patient00001/study1/view1_frontal.jpg,NaN,1.0
1,data/train/patient00002/study2/view1_frontal.jpg,-1.0,NaN
2,data/train/patient00002/study1/view1_frontal.jpg,NaN,NaN
3,data/train/patient00002/study1/view2_lateral.jpg,NaN,NaN
4,data/train/patient00003/study1/view1_frontal.jpg,NaN,NaN
5,data/train/patient00004/study1/view1_frontal.jpg,0.0,1.0
6,data/train/patient00004/study1/view2_lateral.jpg,0.0,1.0
7,data/train/patient00005/study1/view1_frontal.jpg,0.0,1.0
8,data/train/patient00005/study1/view2_lateral.jpg,0.0,1.0
9,data/train/patient00005/study2/view1_frontal.jpg,NaN,NaN


In [4]:
!ls data/train/patient00004/study1

view1_frontal.jpg  view2_lateral.jpg


In [5]:
# Function to clean "Pleural Effusion" column
def clean_effusion(df):
    df = df.copy()
    df["Pleural Effusion"] = df["Pleural Effusion"].replace("", None)
    df["Pleural Effusion"] = pd.to_numeric(df["Pleural Effusion"], errors='coerce')
    df = df[~((df["Pleural Effusion"].isna()) | (df["Pleural Effusion"] == -1)) | (df["No Finding"] == 1)]
    df.loc[df["No Finding"] == 1, "Pleural Effusion"] = 0
    df = df.drop(columns=["No Finding"])
    return df.reset_index(drop=True)  # Reinitialize index

# Clean train and valid datasets
df_train = clean_effusion(df_train)
df_valid = clean_effusion(df_valid)
df_valid.head(10)



,Path,Pleural Effusion
0,data/valid/patient64541/study1/view1_frontal.jpg,0
1,data/valid/patient64542/study1/view1_frontal.jpg,0
2,data/valid/patient64542/study1/view2_lateral.jpg,0
3,data/valid/patient64543/study1/view1_frontal.jpg,0
4,data/valid/patient64544/study1/view1_frontal.jpg,0
5,data/valid/patient64545/study1/view1_frontal.jpg,1
6,data/valid/patient64546/study1/view1_frontal.jpg,0
7,data/valid/patient64547/study1/view1_frontal.jpg,0
8,data/valid/patient64547/study1/view2_frontal.jpg,0
9,data/valid/patient64547/study1/view3_lateral.jpg,0


In [6]:
df_train.head(10)

,Path,Pleural Effusion
0,data/train/patient00001/study1/view1_frontal.jpg,0.0
1,data/train/patient00004/study1/view1_frontal.jpg,0.0
2,data/train/patient00004/study1/view2_lateral.jpg,0.0
3,data/train/patient00005/study1/view1_frontal.jpg,0.0
4,data/train/patient00005/study1/view2_lateral.jpg,0.0
5,data/train/patient00006/study1/view1_frontal.jpg,0.0
6,data/train/patient00007/study1/view1_frontal.jpg,0.0
7,data/train/patient00007/study2/view1_frontal.jpg,0.0
8,data/train/patient00008/study1/view1_frontal.jpg,1.0
9,data/train/patient00008/study2/view1_frontal.jpg,1.0


In [ ]:
from tqdm import tqdm
import psutil
# Output CSV for embeddings
EMBEDDING_CSV_PATH = "pleural_effusion_embeddings.csv"
# Function to process a single image
def process_image(image_path):
    try:
        if image_path.lower().endswith('.dcm'):
            dicom_data = pydicom.dcmread(image_path)
            img = dicom_data.pixel_array
            img = Image.fromarray(img).convert('L')  # Convert to grayscale
        else:
            img = Image.open(image_path).convert('L')  # Grayscale

        img = img.resize((224, 224))  # Resize
        img_array = np.array(img) / 255.0  # Normalize
        return img_array, os.path.basename(image_path)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# Helper function to process each batch
def process_batch(batch, cxr_client):
    embeddings = []
    for image_path, label in batch:
        result = process_image(image_path)
        if result:
            img_array, image_name = result
            pil_image = Image.fromarray((img_array * 255).astype(np.uint8))
            embedding = cxr_client.get_image_embeddings_from_images([pil_image])

            if hasattr(embedding[0], "general_img_emb"):
                embedding_str = str(embedding[0].general_img_emb[0])
                embeddings.append([embedding_str, label, image_name])
    return embeddings

# Function to process and save embeddings
def process_and_save_embeddings(df, cxr_client):
    with open(EMBEDDING_CSV_PATH, "w") as f:
        f.write("embedding,label,image_name\n")  # CSV header

    # Split the data into batches (for parallel processing)
    batch_size = 32  # Number of images per batch
    batches = [(df.iloc[i:i + batch_size]["Path"].tolist(), df.iloc[i:i + batch_size]["Pleural Effusion"].tolist()) for i in range(0, len(df), batch_size)]

    # Create a progress queue for updating progress
    progress_queue = multiprocessing.Queue()

    # Determine the number of available threads based on system memory
    def get_available_memory():
        memory = psutil.virtual_memory()
        return memory.available / (1024 ** 3)  # Return available memory in GB

    def determine_max_threads():
        available_memory = get_available_memory()
        print(f"Available memory: {available_memory} GB")

        # Start with 3 threads, and adjust based on available memory
        max_threads = int((available_memory - 2) // 0.4)  # Keep 2 GB buffer for the OS

        # Ensure the maximum number of threads doesn't exceed 16 (for safety)
        max_threads = min(max_threads, 16)

        return max_threads

    max_threads = determine_max_threads()

    # Start multiprocessing pool
    with multiprocessing.Pool(processes=max_threads) as pool:
        all_embeddings = []
        for batch in tqdm(pool.imap_unordered(lambda b: process_batch(b, cxr_client), batches, chunksize=1), total=len(batches), desc="Processing images"):
            all_embeddings.extend(batch)
            while not progress_queue.empty():
                progress_queue.get()  # Update progress

    # Write embeddings to CSV file
    df_out = pd.DataFrame(all_embeddings, columns=["embedding", "label", "image_name"])
    df_out.to_csv(EMBEDDING_CSV_PATH, mode="a", header=False, index=False)

# Process train and validation datasets
process_and_save_embeddings(df_train, cxr_client)
process_and_save_embeddings(df_valid, cxr_client)

print(f"✅ Saved embeddings to '{EMBEDDING_CSV_PATH}'")